# Example: Inferring causal structure using a pretrained AVICI model

Amortized variational inference for causal discovery (AVICI) allows inferring causal structure from data based on a **simulator** of the domain of interest. 
By training a neural network to infer structure from the simulated data, it can acquire realistic inductive biases from prior knowledge that is hard to cast as score functions or conditional independence tests.


In this example, we **download a pretrained model checkpoint and perform predictions** for a simulated dataset. Inferring causal structure with a pretrained AVICI model takes only a few seconds since it amounts to one forward pass through the neural network.

Setup for Google Colab (Skip this if running locally).


In [ ]:
%pip install --quiet avici

**Simulate some data:**

In [ ]:
import avici
from avici import simulate_data

# g: [d, d] causal graph of `d` variables
# x: [n, d] data matrix containing `n` observations of the `d` variables
# interv: [n, d] binary matrix indicating which nodes were intervened upon
g, x, interv = simulate_data(n=100, n_interv=20, d=20, domain="linear-gaussian-scm")

**Download and initialize a pretrained model:**

In [ ]:
# load pretrained model
model = avici.load_pretrained(download="linear")

**Predict the causal structure:**

In [ ]:
%%time
# g: [d, d] predicted edge probabilities of the causal graph
g_prob = model(x=x, interv=interv)

In [ ]:
from avici.metrics import shd

# compute SHD and visualize predictions
g_pred = (g_prob > 0.5).astype(int)
print(f"SHD:\n{shd(g, g_pred)}")

avici.visualize(g_prob, true=g)